In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPEN_API_KEY")

In [44]:
output_dir = "Prompts"
output_path = os.path.join(output_dir, "prompt3.txt")

promt_txt = ""
# 프롬프트 데이터를 파일에 저장
with open(output_path, "r", encoding="utf-8") as file:
    promt_txt = file.read()

promt_txt


'You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n\nGive a binary score 1 or 0, where 1 means that the answer is grounded in / supported by the set of facts.'

### 가져온 프롬프트 적용, 이때 프롬프트가 요구하는 사항들을 맞춰줘야 함

- prompt1 : 한국어 답변 최적화 프롬프트
- prompt2 : 질문, 답변 쌍 생성 프롬프트
- prompt3 : LLM 의존여부 확인 프롬프트

In [45]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


# 프롬프트 템플릿 정의 (prompt1, prompt3)
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", promt_txt),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])


# # 프롬프트 템플릿 정의 (prompt2)
# contextual_prompt = ChatPromptTemplate.from_messages([
#     ("system", promt_txt),
#     ("user", "Context: {context}\\n\\nNumberOfPairs: {number_of_pairs}")
# ])

In [ ]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("../data/초거대 언어모델 연구 동향.pdf")

# 페이지 별 문서 로드
docs = loader.load()

In [19]:
import re

def remove_table_section(text):
    # "표"라는 단어가 포함된 부분부터 그 이후의 내용 제거
    text_without_table = re.split(r"\n표", text, maxsplit=1)[0]
    return text_without_table.strip()

for doc in docs:
    doc.page_content = remove_table_section(doc.page_content)

docs[0]

Document(metadata={'source': '../3rd/data/초거대 언어모델 연구 동향.pdf', 'page': 0}, page_content='8 특집원고  초거대 언어모델 연구 동향\n초거대 언어모델 연구 동향\n업스테이지  박찬준*･이원성･김윤기･김지후･이활석\n \n1. 서  론1)\nChatGPT1)와 같은 초거대 언어모델(Large Language \nModel, LLM) 의 등장으로 기존에 병렬적으로 연구되\n던 다양한 자연언어처리 하위 분야들이 하나의 모델\n로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 \n발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의\n응답, 형태소분석 등의 작업을 모두 처리할 수 있게 \n되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하\n느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이\n에 따라 사용자의 목적에 맞는 출력을 생성하는 패러\n다임을 맞이하게 되었다 [1].\nLLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난 \n발전을 이루고 있다. 이러한 발전은 몇 가지 주요한 \n요인에 기반하고 있으며, 이 요인들은 현대 자연언어\n처리 (Natural Language Processing, NLP) 연구의 핵심\n적인 추세로 간주된다. 첫째로, 데이터의 양적 확대는 \n무시할 수 없는 중요한 요인이다. 디지털화의 선도로, \n텍스트 데이터의 양이 기하급수적으로 증가하였고, \n이는 연구의 질적 변화를 가져왔다. 대규모 코퍼스의 \n활용은 LLM의 일반화 능력을 향상시키며, 다양한 맥\n락과 주제에 대한 깊은 학습을 가능하게 한다. 둘째\n로, 컴퓨팅 기술의 진보는 LLM의 발전에 있어 결정\n적이었다. 특히, Graphics Processing Unit (GPU) 및 \nTensor Processing Unit (TPU) 와 같은 고성능 병렬 처\n리 하드웨어의 개발은 모델 학습에 있어 병목 현상을 \n크게 완화시켰다. 이로 인해 연구자들은 모델의 복잡\n성을 키우고,

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

# 문서 분할
splits_recur = recursive_text_splitter.split_documents(docs)
splits = splits_recur

In [21]:
from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.vectorstores import FAISS

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

In [22]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(docs)
faiss_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, faiss_retriever],
            weights=[0.5, 0.5]  # 가중치 설정 (가중치의 합은 1.0)
        )

### RAG 체인 구성

- prompt2의 경우에는 질문/답변 쌍의 갯수를 받는 input이 존재해야 함
- 나머지 사항들은 필수과제와 구현사항 동일

In [46]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(temperature=0, model="gpt-4o-mini", api_key=api_key)

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
    
    
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}


# class ContextToText(RunnablePassthrough):
#     def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
#         # context의 각 문서를 문자열로 결합
#         context_text = "\n".join([doc.page_content for doc in inputs["context"]])
#         return {"context": context_text, "data_format": str, "number_of_pairs": inputs["number_of_pairs"]}
    

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model

# # RAG 체인에서 각 단계마다 DebugPassThrough 추가
# rag_chain_debug = {
#     "context": retriever,                    # 컨텍스트를 가져오는 retriever
#     "number_of_pairs": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
# }  | DebugPassThrough() | ContextToText()|   contextual_prompt | model


### 챗봇 구동 확인
- Results 디렉토리 결과 참고
- Prompt1 : 존재하지 않은 질문 물어봤을 때 대응 프롬프트 존재 
- Prompt2 : 자료를 근거로 한 질문/답변 쌍 생성하는 프롬프트 존재
- Prompt3 : 자료를 근거로 한 문장인지를 판별하는 프롬프트 존재

In [50]:
import datetime

while True:
    # number_of_pairs만 입력받기
    # number_of_pairs = (input("몇 개의 질문/답변 쌍을 생성할까요? : "))  # number_of_pairs 입력받기

    # if int(number_of_pairs) < 0:
    #     print("유효한 숫자를 입력해주세요.")
    #     continue

    # if int(number_of_pairs) == 0:
    #     break

    # # inputs 구성
    # inputs = {"context" : splits, "number_of_pairs": number_of_pairs}
    
    # response = rag_chain_debug.invoke(number_of_pairs)

    inputs= input("질문을 입력하세요! 종료를 원한다면 exit을 입력하세요.")
    if inputs == "exit":
        break
    print("question: " + inputs)
    
    response = rag_chain_debug.invoke(inputs)
    print("RAG response : " + response.content)

    # 출력 파일 경로 설정
    output_dir = "Results"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H%M%S")
    output_path = os.path.join(output_dir, f"prompt3_result_{timestamp}.txt")

    # 프롬프트 데이터를 파일에 저장
    with open(output_path, "w", encoding="utf-8") as file:
        file.write(f"Reliability (if answer is based on your given data, it will return 1): \n{response.content}")

    print(f"Prompt saved to {output_path}")

question: What is the significance of the Pretrain-Finetune paradigm in LLM development?
Debug Output: What is the significance of the Pretrain-Finetune paradigm in LLM development?
Debug Output: {'context': [Document(metadata={'source': '../3rd/data/초거대 언어모델 연구 동향.pdf', 'page': 10}, page_content='18 특집원고  초거대 언어모델 연구 동향\n버시 보호, 3) 다양성 존중, 4) 침해금지, 5) 공공성, 6) \n연대성, 7) 데이터 관리, 8) 책임성, 9) 안정성, 10) 투\n명성 등의 요건이 충족되어야 한다는 내용이 포함되\n어 있다.\n다양한 윤리 원칙에 명시된 내용들은 크게 6가지로 \n인간성, 책임성, 보안성, 안전성, 투명성, 다양성으로 \n구분된다.\n인간성 (Humanity & Human-centered)은 인공지능\n의 개발과 활용은 인간과 사회에 유익한 가치를 제공\n하며 인간의 권리와 자유를 침해하지 않는다는 내용\n이다.\n책임성 (Responsibility & Accountability)은 인공지\n능을 개발하고 활용하는 주체들의 역할과 책임을 명\n확히 설정하여 발생할 수 있는 피해를 최소화 한다는 \n내용이다.\n보안성 (Privacy & Security)은 인공지능 개발 및 활\n용하는 과정에서 사용자의 개인정보와 프라이버시를 \n보호하기 위해 정보 보안을 고려하여 설계한다는 내\n용이다.\n안전성 (Safety & Reliability)은 인공지능의 개발과 \n활용 과정에서 발생할 수 있는 잠재적 위험에 대응하\n고 안전하게 작동할 수 있도록 한다는 내용이다.\n투명성 (Transparency & Explainability)은 인공지능 \n의 작동 방식 또는 데이터 활용 방안에 대해 투명하\n게 